# Dodanie ścieżki do projektu

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Importowanie bibliotek

In [2]:
from src.scrapers import healthdata, koronawirusunas, meteostat, policjapl, rmf24
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from src.utils import save, paths
from pathlib import Path

# Pobranie i zapisanie danych

In [ ]:
update = False

if len(os.listdir(paths.get_path())) == 0 or update:
    data = {**healthdata.get_data(),
            **koronawirusunas.get_data(),
            **{"meteostat": pd.DataFrame(meteostat.get_data(end_date=str(datetime.datetime.now().date()), start_date="2020-01-01", key="cKAhL60A" ))},
            **policjapl.scrape() 
           }
    save.save_dataframes(data=data)
else:
    print("Data directory is not empty. propably data inside.")

In [44]:
p = policjapl.scrape()

ConnectionResetError: [Errno 104] Connection reset by peer

# Załadowanie danych

In [4]:
policja = pd.read_csv(Path(paths.get_path(), "policjapl.csv"), index_col=1, parse_dates=True, dayfirst=True)
health = pd.read_csv(Path(paths.get_path(), "healthdata.csv"), index_col=2, parse_dates=True, dayfirst=True)
korona = pd.read_csv(Path(paths.get_path(), "koronawirusunas.csv"), index_col=0, parse_dates=True, dayfirst=True)
meteo = pd.read_csv(Path(paths.get_path(), "meteostat.csv"), index_col=1, parse_dates=True, dayfirst=True)

# Czyszczenie danych

## Czyszczenie meteo

In [5]:
meteo.head()

,Unnamed: 0,temperature,temperature_min,temperature_max,precipitation,snowfall,snowdepth,winddirection,windspeed,peakgust,sunshine,pressure
date,,,,,,,,,,,,
2020-02-01,0,9.4,5.9,11.6,2.8,NaN,NaN,221.0,9.6,NaN,NaN,1003.3
2020-02-02,1,7.1,4.7,10.9,3.8,NaN,NaN,256.0,20.0,NaN,NaN,1001.8
2020-02-03,2,3.9,2.1,6.0,0.5,NaN,NaN,251.0,13.4,NaN,NaN,1004.9
2020-02-04,3,4.9,1.6,6.1,1.5,NaN,NaN,293.0,12.3,NaN,NaN,999.6
2020-02-05,4,0.6,-1.2,3.4,2.3,NaN,NaN,247.0,19.5,NaN,NaN,1019.9


In [6]:
meteo.drop(["snowfall", "snowdepth", "peakgust", "sunshine","precipitation","Unnamed: 0"], axis=1, inplace=True)
meteo.index.name = None
meteo.head()

,temperature,temperature_min,temperature_max,winddirection,windspeed,pressure
2020-02-01,9.4,5.9,11.6,221.0,9.6,1003.3
2020-02-02,7.1,4.7,10.9,256.0,20.0,1001.8
2020-02-03,3.9,2.1,6.0,251.0,13.4,1004.9
2020-02-04,4.9,1.6,6.1,293.0,12.3,999.6
2020-02-05,0.6,-1.2,3.4,247.0,19.5,1019.9


In [7]:
meteo[meteo.isnull().any(axis=1)]

,temperature,temperature_min,temperature_max,winddirection,windspeed,pressure
2020-05-03,10.4,6.0,14.2,NaN,11.4,1011.8


In [8]:
meteo['winddirection'].fillna(meteo['winddirection'].mean(), inplace = True)

In [9]:
meteo[meteo.isnull().any(axis=1)]

,temperature,temperature_min,temperature_max,winddirection,windspeed,pressure


## Czyszczenie health

In [10]:
health.columns

Index(['V1', 'location_name', 'allbed_mean', 'allbed_lower', 'allbed_upper',
       'ICUbed_mean', 'ICUbed_lower', 'ICUbed_upper', 'InvVen_mean',
       'InvVen_lower', 'InvVen_upper', 'deaths_mean', 'deaths_lower',
       'deaths_upper', 'admis_mean', 'admis_lower', 'admis_upper',
       'newICU_mean', 'newICU_lower', 'newICU_upper', 'totdea_mean',
       'totdea_lower', 'totdea_upper', 'bedover_mean', 'bedover_lower',
       'bedover_upper', 'icuover_mean', 'icuover_lower', 'icuover_upper',
       'deaths_mean_smoothed', 'deaths_lower_smoothed',
       'deaths_upper_smoothed', 'totdea_mean_smoothed',
       'totdea_lower_smoothed', 'totdea_upper_smoothed', 'mobility_data_type',
       'mobility_composite', 'total_tests_data_type', 'total_tests',
       'confirmed_infections', 'est_infections_mean', 'est_infections_lower',
       'est_infections_upper'],
      dtype='object')

In [11]:
health = health.drop(health[health.index > "2020-05-20"].index)
health = health[health.location_name == "Poland"]
health = health[["allbed_mean", "ICUbed_mean", "deaths_mean", "admis_mean", "newICU_mean", 
                 "totdea_mean", "bedover_mean", "icuover_mean", 
                 "mobility_composite", "total_tests", "confirmed_infections"]]
health.index.name = None

In [12]:
health.tail()

,allbed_mean,ICUbed_mean,deaths_mean,admis_mean,newICU_mean,totdea_mean,bedover_mean,icuover_mean,mobility_composite,total_tests,confirmed_infections
2020-05-16,531.896330,162.255641,16.660,67.499215,20.402642,912.462,0.0,0.0,-45.705095,14914.884465,NaN
2020-05-17,527.067489,160.814884,16.280,67.230701,20.406182,928.742,0.0,0.0,-45.706528,15113.480201,NaN
2020-05-18,523.575463,159.716028,15.989,66.909504,20.335671,944.731,0.0,0.0,-45.707487,15312.075937,NaN
2020-05-19,519.898537,158.514489,15.859,65.460435,19.919147,960.590,0.0,0.0,-45.708130,15510.671673,NaN
2020-05-20,514.962577,156.822654,15.876,63.071912,19.363049,976.466,0.0,0.0,-45.708560,15709.267409,NaN


In [13]:
health[health.isnull().any(axis=1)]

,allbed_mean,ICUbed_mean,deaths_mean,admis_mean,newICU_mean,totdea_mean,bedover_mean,icuover_mean,mobility_composite,total_tests,confirmed_infections
2019-12-18,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.0,NaN,NaN,NaN
2019-12-19,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.0,NaN,NaN,NaN
2019-12-20,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.0,NaN,NaN,NaN
2019-12-21,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.0,NaN,NaN,NaN
2019-12-22,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2020-05-16,531.896330,162.255641,16.660,67.499215,20.402642,912.462,0.0,0.0,-45.705095,14914.884465,NaN
2020-05-17,527.067489,160.814884,16.280,67.230701,20.406182,928.742,0.0,0.0,-45.706528,15113.480201,NaN
2020-05-18,523.575463,159.716028,15.989,66.909504,20.335671,944.731,0.0,0.0,-45.707487,15312.075937,NaN
2020-05-19,519.898537,158.514489,15.859,65.460435,19.919147,960.590,0.0,0.0,-45.708130,15510.671673,NaN


In [14]:
health['mobility_composite'].fillna(0, inplace = True)
health['total_tests'].fillna(0, inplace = True)
health['confirmed_infections'].fillna(0, inplace = True)

In [15]:
health[health.isnull().any(axis=1)]

,allbed_mean,ICUbed_mean,deaths_mean,admis_mean,newICU_mean,totdea_mean,bedover_mean,icuover_mean,mobility_composite,total_tests,confirmed_infections


## Czyszczenie policja

policja.head()

In [16]:
policja.drop(["Unnamed: 0"], axis=1, inplace=True)
policja.index.name = None
policja.head()

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach
2020-05-17,14699,753,151,275,48,5,55
2020-05-16,17644,655,168,245,62,2,69
2020-05-15,17037,554,262,160,56,6,56
2020-05-14,15485,608,304,174,38,1,45
2020-05-13,15075,546,272,149,36,2,39


## Czyszczenie korona

In [17]:
korona.head()

,smp,testy,testyl,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
2020-02-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.53,4.70,NaN,NaN,NaN,NaN
2020-02-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.05,6.90,NaN,NaN,NaN,NaN
2020-02-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.24,4.46,NaN,NaN,NaN,NaN
2020-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.95,7.81,NaN,NaN,NaN,NaN
2020-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.57,22.21,NaN,NaN,NaN,NaN


In [18]:
korona.tail()

,smp,testy,testyl,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
2020-05-14,17615.0,554357.0,506733.0,17615.0,10036.0,883.0,6696.0,-48.67,-22.08,2538.0,96708.0,NaN,18256.0
2020-05-15,18016.0,576276.0,526895.0,18016.0,10191.0,907.0,6918.0,-39.24,-12.05,2579.0,95068.0,NaN,18363.0
2020-05-16,18257.0,601394.0,549792.0,18257.0,10167.0,915.0,7175.0,NaN,NaN,2407.0,90386.0,NaN,18355.0
2020-05-17,18529.0,620249.0,576055.0,18529.0,10153.0,925.0,7451.0,NaN,NaN,2459.0,84542.0,NaN,18195.0
2020-05-18,NaN,NaN,NaN,18529.0,9976.0,925.0,7628.0,NaN,NaN,2544.0,80947.0,NaN,18772.0


### Wypełnienie nan

In [19]:
korona['kwar_z'].fillna(0, inplace=True)
korona.fillna(method='ffill', inplace=True)
korona.fillna(0, inplace=True)
korona

,smp,testy,testyl,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
2020-02-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.53,4.70,0.0,0.0,0.0,0.0
2020-02-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.05,6.90,0.0,0.0,0.0,0.0
2020-02-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.24,4.46,0.0,0.0,0.0,0.0
2020-02-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.95,7.81,0.0,0.0,0.0,0.0
2020-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.57,22.21,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-14,17615.0,554357.0,506733.0,17615.0,10036.0,883.0,6696.0,-48.67,-22.08,2538.0,96708.0,0.0,18256.0
2020-05-15,18016.0,576276.0,526895.0,18016.0,10191.0,907.0,6918.0,-39.24,-12.05,2579.0,95068.0,0.0,18363.0
2020-05-16,18257.0,601394.0,549792.0,18257.0,10167.0,915.0,7175.0,-39.24,-12.05,2407.0,90386.0,0.0,18355.0
2020-05-17,18529.0,620249.0,576055.0,18529.0,10153.0,925.0,7451.0,-39.24,-12.05,2459.0,84542.0,0.0,18195.0


In [20]:
korona[korona.isnull().any(axis=1)]

,smp,testy,testyl,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor


## Łączenie danych

In [21]:
df = pd.merge(policja, meteo, how='outer', left_index=True, right_index=True)
df = pd.merge(df, health, how='outer', left_index=True, right_index=True)
df = pd.merge(df, korona, how='outer', left_index=True, right_index=True)

In [39]:
df.iloc[0]

Interwencje                        NaN
Zatrzymani na gorącym uczynku      NaN
Zatrzymani poszukiwani             NaN
Zatrzymani nietrzeźwi kierujący    NaN
Wypadki drogowe                    NaN
Zabici w wypadkach                 NaN
Ranni w wypadkach                  NaN
temperature                        NaN
temperature_min                    NaN
temperature_max                    NaN
winddirection                      NaN
windspeed                          NaN
pressure                           NaN
allbed_mean                        0.0
ICUbed_mean                        0.0
deaths_mean                        0.0
admis_mean                         0.0
newICU_mean                        0.0
totdea_mean                        0.0
bedover_mean                       0.0
icuover_mean                       0.0
mobility_composite                 0.0
total_tests                        0.0
confirmed_infections               0.0
smp                                NaN
testy                    

## Czyszczenie df

In [32]:
df.fillna(method='ffill', inplace=True)

In [35]:
df.head()

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,temperature,temperature_min,temperature_max,...,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
2019-12-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-20,18837.0,550.0,280.0,169.0,105.0,7.0,115.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-21,19185.0,555.0,195.0,220.0,100.0,11.0,118.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-22,15600.0,534.0,201.0,207.0,74.0,2.0,86.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Usunięcie danych sprzed 01.01.2020

In [40]:
df.drop(df[df.index < "2020-01-01"].index, inplace = True)

In [41]:
df

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,temperature,temperature_min,temperature_max,...,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
2020-01-01,19145.0,734.0,188.0,332.0,42.0,1.0,46.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02,14654.0,513.0,333.0,166.0,60.0,5.0,67.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,15507.0,468.0,405.0,122.0,62.0,3.0,69.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-04,14528.0,435.0,176.0,135.0,59.0,4.0,64.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-05,11948.0,457.0,185.0,146.0,44.0,4.0,54.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-16,17644.0,655.0,168.0,245.0,62.0,2.0,69.0,10.1,5.0,14.0,...,18257.0,10167.0,915.0,7175.0,-39.24,-12.05,2407.0,90386.0,0.0,18355.0
2020-05-17,14699.0,753.0,151.0,275.0,48.0,5.0,55.0,11.8,5.0,18.0,...,18529.0,10153.0,925.0,7451.0,-39.24,-12.05,2459.0,84542.0,0.0,18195.0
2020-05-18,14699.0,753.0,151.0,275.0,48.0,5.0,55.0,11.8,5.0,18.0,...,18529.0,9976.0,925.0,7628.0,-39.24,-12.05,2544.0,80947.0,0.0,18772.0
2020-05-19,14699.0,753.0,151.0,275.0,48.0,5.0,55.0,11.8,5.0,18.0,...,18529.0,9976.0,925.0,7628.0,-39.24,-12.05,2544.0,80947.0,0.0,18772.0


# Współcznniki korelacji

In [23]:
def highlight(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    l = []
    for v in s:
        if abs(v) > 0.85 and abs(v) != 1:
            l.append('background-color: red')
        elif abs(v) > 0.75 and abs(v) != 1:
            l.append('background-color: yellow')
        else: 
            l.append('')
    return l

## Pearson

In [24]:
cor_pearson = df.corr(method="pearson")
cor_pearson.style.apply(highlight)

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,temperature,temperature_min,temperature_max,winddirection,windspeed,pressure,allbed_mean,ICUbed_mean,deaths_mean,admis_mean,newICU_mean,totdea_mean,bedover_mean,icuover_mean,mobility_composite,total_tests,confirmed_infections,smp,testy,testyl,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
Interwencje,1.000000,0.218204,0.081724,0.238929,0.262340,0.122284,0.228177,0.490110,0.384253,0.457240,-0.088139,-0.279728,-0.006365,0.344681,0.339522,0.349589,0.328048,0.334625,0.281557,nan,nan,-0.169134,0.306797,0.280627,0.318368,0.286615,0.191474,0.318368,0.378072,0.291028,0.237012,0.055442,0.092250,0.307596,0.286064,-0.352100,-0.185709
Zatrzymani na gorącym uczynku,0.218204,1.000000,0.547149,0.623770,0.558776,0.177899,0.555380,0.042177,0.354528,-0.077870,0.135490,-0.090068,-0.536533,-0.247262,-0.226447,-0.223195,-0.342020,-0.325792,0.080862,nan,nan,0.595932,-0.166373,-0.482080,0.341559,0.408933,0.491454,0.341559,0.310093,0.416820,0.498520,0.725276,0.824631,-0.174442,-0.407270,-0.521049,-0.832793
Zatrzymani poszukiwani,0.081724,0.547149,1.000000,-0.192743,0.643283,0.039188,0.623077,-0.233921,0.065125,-0.319069,0.146420,0.057671,-0.499165,-0.457695,-0.447938,-0.363078,-0.511074,-0.501242,-0.263971,nan,nan,0.678352,-0.440971,-0.557416,-0.120112,-0.070486,0.060905,-0.120112,-0.126713,-0.049023,0.032016,0.788148,0.816601,-0.479169,-0.564356,-0.410557,-0.727544
Zatrzymani nietrzeźwi kierujący,0.238929,0.623770,-0.192743,1.000000,0.163585,0.211720,0.182184,0.271501,0.387221,0.202993,0.036444,-0.181043,-0.256181,0.094953,0.108568,0.029799,0.033902,0.044489,0.262308,nan,nan,0.133627,0.145208,-0.052712,0.429587,0.455787,0.433775,0.429587,0.422931,0.456943,0.469632,0.247247,0.320793,0.133993,-0.050866,-0.352060,-0.455473
Wypadki drogowe,0.262340,0.558776,0.643283,0.163585,1.000000,0.389475,0.977048,-0.014038,0.272891,-0.113152,0.100063,-0.123980,-0.551088,-0.401804,-0.388721,-0.344993,-0.472340,-0.459428,-0.166421,nan,nan,0.630597,-0.361367,-0.502289,0.058254,0.116678,0.232007,0.058254,0.034205,0.141339,0.216876,0.772088,0.825967,-0.342335,-0.571242,-0.363108,-0.734112
Zabici w wypadkach,0.122284,0.177899,0.039188,0.211720,0.389475,1.000000,0.346482,0.015250,0.126363,-0.042351,0.034029,0.051009,-0.198118,-0.121375,-0.113777,-0.123114,-0.143061,-0.140136,-0.068267,nan,nan,0.194044,-0.116875,-0.106738,-0.026784,-0.028949,-0.018140,-0.026784,0.000719,-0.001711,-0.018048,0.232960,0.197285,-0.139114,-0.220713,-0.034646,-0.214608
Ranni w wypadkach,0.228177,0.555380,0.623077,0.182184,0.977048,0.346482,1.000000,-0.055384,0.238604,-0.131767,0.093027,-0.122941,-0.544777,-0.404487,-0.390403,-0.344541,-0.480926,-0.466912,-0.160533,nan,nan,0.645164,-0.362634,-0.514572,0.059763,0.117204,0.240779,0.059763,0.037737,0.145715,0.218578,0.784887,0.832134,-0.348740,-0.583307,-0.395852,-0.756082
temperature,0.490110,0.042177,-0.233921,0.271501,-0.014038,0.015250,-0.055384,1.000000,0.802625,0.961456,-0.067656,-0.216670,-0.147625,0.685844,0.691148,0.672836,0.623249,0.638021,0.660456,nan,nan,-0.469582,0.702584,0.515313,0.662201,0.619439,0.469041,0.662201,0.705864,0.641774,0.542488,-0.172727,-0.019669,0.617883,0.390028,-0.270124,-0.143064
temperature_min,0.384253,0.354528,0.065125,0.387221,0.272891,0.126363,0.238604,0.802625,1.000000,0.674579,0.043487,-0.045153,-0.411329,0.437541,0.455051,0.449078,0.353352,0.369185,0.578098,nan,nan,-0.124003,0.502096,0.229696,0.605541,0.594397,0.551003,0.605541,0.609633,0.621112,0.572607,0.094443,0.254966,0.408744,0.176345,-0.314188,-0.347969
temperature_max,0.457240,-0.077870,-0.319069,0.202993,-0.113152,-0.042351,-0.131767,0.961456,0.674579,1.000000,-0.076110,-0.253848,-0.063451,0.694827,0.695856,0.681902,0.644608,0.657737,0.624408,nan,nan,-0.541409,0.698704,0.554365,0.607765,0.560935,0.411022,0.607765,0.656841,0.579255,0.478036,-0.247859,-0.113880,0.623387,0.420880,-0.213555,-0.059

## Kendall

In [25]:
cor_pearson = df.corr(method="kendall")
cor_pearson.style.apply(highlight)

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,temperature,temperature_min,temperature_max,winddirection,windspeed,pressure,allbed_mean,ICUbed_mean,deaths_mean,admis_mean,newICU_mean,totdea_mean,bedover_mean,icuover_mean,mobility_composite,total_tests,confirmed_infections,smp,testy,testyl,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
Interwencje,1.000000,0.123751,0.042397,0.178960,0.173197,0.111606,0.158064,0.337965,0.259879,0.307775,-0.078664,-0.190587,-0.013597,0.177213,0.170450,0.215026,0.192853,0.191092,0.164959,nan,nan,-0.103189,0.173878,0.160232,0.194984,0.195244,0.162588,0.194984,0.277443,0.188524,0.279397,0.089029,0.067203,0.203637,0.217482,-0.328166,-0.042689
Zatrzymani na gorącym uczynku,0.123751,1.000000,0.398097,0.431428,0.414550,0.117697,0.409715,-0.042346,0.181287,-0.117783,0.007788,-0.063436,-0.332773,-0.339409,-0.329464,-0.297093,-0.371572,-0.367091,-0.252146,nan,nan,0.469090,-0.235550,-0.416296,0.122745,0.123016,0.370720,0.122745,0.300300,0.125943,0.311386,0.617398,0.640978,-0.018773,-0.335887,-0.447277,-0.679572
Zatrzymani poszukiwani,0.042397,0.398097,1.000000,-0.081196,0.512319,0.043214,0.492313,-0.110589,0.072569,-0.172558,0.075248,-0.003013,-0.337283,-0.405538,-0.405114,-0.335910,-0.428185,-0.425831,-0.376009,nan,nan,0.472730,-0.352390,-0.428703,-0.078035,-0.078024,0.132811,-0.078035,0.077005,-0.070215,0.068522,0.596259,0.624461,-0.105492,-0.336707,-0.381730,-0.574280
Zatrzymani nietrzeźwi kierujący,0.178960,0.431428,-0.081196,1.000000,0.136614,0.178840,0.156569,0.137466,0.202169,0.102619,-0.019695,-0.134103,-0.178129,-0.045315,-0.038736,-0.043659,-0.066328,-0.062625,-0.010678,nan,nan,0.159636,-0.004440,-0.118250,0.183029,0.183292,0.284348,0.183029,0.313549,0.181632,0.320742,0.251443,0.247406,0.105416,-0.094411,-0.338847,-0.329569
Wypadki drogowe,0.173197,0.414550,0.512319,0.136614,1.000000,0.245241,0.869119,-0.016934,0.164005,-0.079652,0.012109,-0.077904,-0.384602,-0.385337,-0.380650,-0.360414,-0.417714,-0.414277,-0.363366,nan,nan,0.458449,-0.331324,-0.411537,-0.052616,-0.052899,0.193696,-0.052616,0.096816,-0.062644,0.092878,0.668612,0.656456,-0.081105,-0.406687,-0.332825,-0.575874
Zabici w wypadkach,0.111606,0.117697,0.043214,0.178840,0.245241,1.000000,0.219327,0.003199,0.061795,-0.030475,0.029516,0.060600,-0.140850,-0.110367,-0.107468,-0.105955,-0.117279,-0.118416,-0.116584,nan,nan,0.135376,-0.107285,-0.067346,-0.048610,-0.048909,0.004215,-0.048610,-0.004487,-0.056249,-0.005177,0.151289,0.131807,-0.040050,-0.151334,-0.019751,-0.158060
Ranni w wypadkach,0.158064,0.409715,0.492313,0.156569,0.869119,0.219327,1.000000,-0.053250,0.142284,-0.108778,0.009074,-0.086740,-0.378571,-0.402295,-0.394421,-0.368247,-0.436130,-0.432061,-0.368626,nan,nan,0.484061,-0.334409,-0.428646,-0.060430,-0.060131,0.205598,-0.060430,0.100696,-0.066447,0.094792,0.674008,0.656652,-0.087740,-0.432308,-0.349836,-0.589775
temperature,0.337965,-0.042346,-0.110589,0.137466,-0.016934,0.003199,-0.053250,1.000000,0.601085,0.836912,-0.051432,-0.145939,-0.076535,0.462171,0.461794,0.474476,0.425258,0.434239,0.472679,nan,nan,-0.242134,0.477026,0.345175,0.453481,0.453126,0.363933,0.453481,0.507719,0.453605,0.514254,0.024204,0.008068,0.484157,0.254956,-0.214482,-0.007235
temperature_min,0.259879,0.181287,0.072569,0.202169,0.164005,0.061795,0.142284,0.601085,1.000000,0.487164,0.028942,-0.044259,-0.264102,0.236066,0.248511,0.260179,0.192700,0.192946,0.301352,nan,nan,-0.049690,0.308084,0.101829,0.365190,0.364847,0.450254,0.365190,0.460351,0.361498,0.461659,0.188294,0.202135,0.350131,0.099334,-0.264529,-0.203012
temperature_max,0.307775,-0.117783,-0.172558,0.102619,-0.079652,-0.030475,-0.108778,0.836912,0.487164,1.000000,-0.057796,-0.163990,-0.022315,0.491377,0.489117,0.497334,0.454853,0.464970,0.487384,nan,nan,-0.288296,0.490344,0.391148,0.424296,0.423946,0.312332,0.424296,0.463345,0.425707,0.463998,-0.042909,-0.052700,0.467102,0.275055

## Pearson

In [26]:
cor_pearson = df.corr(method="spearman")
cor_pearson.style.apply(highlight)

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,temperature,temperature_min,temperature_max,winddirection,windspeed,pressure,allbed_mean,ICUbed_mean,deaths_mean,admis_mean,newICU_mean,totdea_mean,bedover_mean,icuover_mean,mobility_composite,total_tests,confirmed_infections,smp,testy,testyl,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
Interwencje,1.000000,0.183706,0.110949,0.244235,0.250018,0.154609,0.220614,0.494271,0.379449,0.454773,-0.098061,-0.287921,-0.019209,0.256876,0.250603,0.307635,0.271674,0.267247,0.251044,nan,nan,-0.144958,0.260637,0.222888,0.333283,0.333362,0.207688,0.333283,0.406700,0.327098,0.409402,0.139523,0.109553,0.348864,0.295090,-0.404655,-0.124740
Zatrzymani na gorącym uczynku,0.183706,1.000000,0.566795,0.596424,0.594662,0.161258,0.587858,-0.046802,0.263824,-0.161933,0.025327,-0.104736,-0.486613,-0.480704,-0.470603,-0.416754,-0.513701,-0.510113,-0.397405,nan,nan,0.647144,-0.374778,-0.571505,0.061411,0.061476,0.447913,0.061411,0.315009,0.060816,0.319876,0.830423,0.845095,-0.070574,-0.459584,-0.550824,-0.866242
Zatrzymani poszukiwani,0.110949,0.566795,1.000000,-0.099913,0.715100,0.066609,0.693787,-0.161223,0.120791,-0.259027,0.108547,0.002307,-0.484103,-0.570562,-0.569560,-0.482528,-0.589375,-0.586602,-0.551756,nan,nan,0.651708,-0.524121,-0.595073,-0.181355,-0.181354,0.162740,-0.181355,0.056882,-0.177986,0.049239,0.794947,0.818601,-0.193300,-0.470570,-0.467238,-0.755348
Zatrzymani nietrzeźwi kierujący,0.244235,0.596424,-0.099913,1.000000,0.204483,0.251413,0.231484,0.193101,0.303990,0.140573,-0.032840,-0.205479,-0.264403,-0.056340,-0.047410,-0.058406,-0.085263,-0.080210,-0.015276,nan,nan,0.232179,-0.006519,-0.157065,0.261119,0.261194,0.357021,0.261119,0.411904,0.256409,0.419730,0.352669,0.343139,0.172475,-0.123069,-0.420346,-0.465669
Wypadki drogowe,0.250018,0.594662,0.715100,0.204483,1.000000,0.341495,0.969391,-0.038134,0.247966,-0.135765,0.027120,-0.122591,-0.545669,-0.543525,-0.538503,-0.493661,-0.578215,-0.574981,-0.517883,nan,nan,0.630294,-0.483666,-0.562810,-0.122802,-0.122882,0.245667,-0.122802,0.101934,-0.127213,0.100508,0.860013,0.849367,-0.158567,-0.562901,-0.415258,-0.787445
Zabici w wypadkach,0.154609,0.161258,0.066609,0.251413,0.341495,1.000000,0.305602,0.003536,0.087187,-0.043789,0.040450,0.091402,-0.214860,-0.146991,-0.142627,-0.133970,-0.158887,-0.156318,-0.149874,nan,nan,0.185283,-0.141352,-0.093216,-0.066661,-0.067003,0.004622,-0.066661,-0.003309,-0.075509,-0.005028,0.211736,0.191392,-0.047077,-0.220659,-0.020901,-0.236682
Ranni w wypadkach,0.220614,0.587858,0.693787,0.231484,0.969391,0.305602,1.000000,-0.085058,0.210248,-0.165944,0.019879,-0.123603,-0.538272,-0.555927,-0.549749,-0.503327,-0.595374,-0.591635,-0.524682,nan,nan,0.655641,-0.488229,-0.579043,-0.130688,-0.130657,0.262161,-0.130688,0.101785,-0.133724,0.098064,0.866950,0.857216,-0.162106,-0.594132,-0.437490,-0.794882
temperature,0.494271,-0.046802,-0.161223,0.193101,-0.038134,0.003536,-0.085058,1.000000,0.797080,0.961781,-0.071839,-0.216799,-0.115726,0.654030,0.655997,0.643027,0.615451,0.624373,0.667198,nan,nan,-0.362881,0.672011,0.490964,0.669585,0.669480,0.472033,0.669585,0.703525,0.667603,0.707127,0.049133,0.044769,0.697033,0.380879,-0.268600,-0.039627
temperature_min,0.379449,0.263824,0.120791,0.303990,0.247966,0.087187,0.210248,0.797080,1.000000,0.681699,0.036256,-0.066245,-0.375735,0.352079,0.365699,0.379687,0.295265,0.298750,0.429762,nan,nan,-0.062902,0.444309,0.161488,0.533377,0.532878,0.579971,0.533377,0.620032,0.522929,0.621844,0.304534,0.314215,0.517307,0.149154,-0.336198,-0.309697
temperature_max,0.454773,-0.161933,-0.259027,0.140573,-0.135765,-0.043789,-0.165944,0.961781,0.681699,1.000000,-0.077955,-0.240737,-0.041575,0.686344,0.684693,0.668071,0.652848,0.662221,0.690276,nan,nan,-0.431057,0.694969,0.545771,0.631448,0.631394,0.409672,0.631448,0.645139,0.631269,0.646725,-0.045507,-0.055857,0.671880,0.407079,

In [27]:
df.describe()

,Interwencje,Zatrzymani na gorącym uczynku,Zatrzymani poszukiwani,Zatrzymani nietrzeźwi kierujący,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,temperature,temperature_min,temperature_max,...,zar,chor,zgo,wyl,pieszo,pojazdem,hosp,kwar,kwar_z,nadzor
count,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,107.000000,107.000000,107.000000,...,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000
mean,14819.693333,496.153333,246.066667,159.980000,53.093333,5.140000,60.260000,7.054206,1.830841,11.883178,...,6333.011765,4223.223529,280.647059,1537.752941,-46.851647,-36.921647,1780.964706,78267.941176,17350.811765,25076.752941
std,1953.381168,111.759914,97.034462,45.076738,19.537727,2.870891,23.132658,4.184434,3.396958,5.290905,...,6335.833471,4241.086635,322.432559,2289.120370,34.098657,26.710867,1083.678660,55780.233033,44458.821799,16997.624585
min,9992.000000,254.000000,86.000000,79.000000,14.000000,0.000000,15.000000,-0.300000,-7.400000,2.200000,...,0.000000,0.000000,0.000000,0.000000,-84.100000,-80.220000,0.000000,0.000000,0.000000,0.000000
25%,13696.500000,401.250000,164.500000,129.250000,37.000000,3.000000,42.250000,3.900000,-0.450000,7.650000,...,177.000000,0.000000,4.000000,0.000000,-72.440000,-60.020000,768.000000,10050.000000,0.000000,17155.000000
50%,14685.500000,518.500000,220.000000,151.500000,54.000000,5.000000,61.500000,6.600000,2.000000,11.600000,...,4413.000000,4144.000000,107.000000,162.000000,-59.130000,-42.650000,2409.000000,94297.000000,0.000000,18874.000000
75%,15822.000000,580.750000,341.000000,178.750000,66.000000,7.000000,74.750000,10.150000,4.150000,16.050000,...,11902.000000,8874.000000,562.000000,2466.000000,-32.600000,-12.050000,2649.000000,106525.000000,0.000000,35853.000000
max,19989.000000,753.000000,415.000000,332.000000,128.000000,15.000000,152.000000,17.500000,9.800000,25.400000,...,18529.000000,10191.000000,925.000000,7628.000000,43.570000,22.210000,3505.000000,183095.000000,169092.000000,59081.000000
